In [13]:
import os

import pandas as pd
from time import time
from pathlib import Path


In [14]:
here_path = Path(os.path.abspath(''))
ROOT_PATH = here_path.parent.parent
ROOT_PATH

WindowsPath('C:/Users/ldecarva/Documents/code_gpd')

In [15]:
DATA_PATH = ROOT_PATH.parent / "Data"
DATA_PATH

WindowsPath('C:/Users/ldecarva/Documents/Data')

In [16]:
GPD_SORTING_OUTPUT_PATH = DATA_PATH / "gpd_sorting_output"
GPD_SORTING_OUTPUT_PATH

WindowsPath('C:/Users/ldecarva/Documents/Data/gpd_sorting_output')

In [17]:
catalogue_path = (
    DATA_PATH / "Danube" / "BDD" / "Present version" /"CATALOGUE-export.csv"
)
catalogue_path

WindowsPath('C:/Users/ldecarva/Documents/Data/Danube/BDD/Present version/CATALOGUE-export.csv')

### Create archetype in the catalogue

In [18]:
catalogue = pd.read_csv(catalogue_path)
initial_cols = catalogue.columns
print(initial_cols)
# create column with Danube archetypes
catalogue["archetype"] = (
    catalogue["NUMERO_PERIODE"].astype(str)
    + "-"
    + catalogue["NOM_TYPOLOGIE"].astype(str)
    + "-"
    + catalogue["USAGE"].astype(str)
    + "-"
    + catalogue["TERRITOIRE"].astype(str)
)
print(set(catalogue.columns) - set(initial_cols), " added")

Index(['NUMERO_TYPOLOGIE', 'NUMERO_PERIODE', 'DISPOSITIF_MUR_OPTION1',
       'DISPOSITIF_MUR_OPTION2', 'DISPOSITIF_MUR_OPTION3',
       'DISPOSTIIF_TOIT_ OPTION1', 'DISPOSITIF_TOIT_OPTION2', 'TERRITOIRE',
       'COUVERTURE_ROUTE', 'VENTILATION', 'COMPACITE', 'POURCENTAGE_VITRAGE',
       'PROTECTIONS_SOLAIRES', 'TYPE_VITRAGE', 'USAGE', 'NOM_TYPOLOGIE',
       'PHOTO', 'source photo', 'PERMEABILITE', 'PLANCHER'],
      dtype='object')
{'archetype'}  added


In [19]:
# list of all Danube archetypes
archs_danube = list(catalogue["archetype"])
print(archs_danube[:2])
print(f"danube_archs len of {len(archs_danube)}")
print(f"danube_archs set len of {len(set(archs_danube))}")
# The list and set have the same size, so there is no double entry

['P1-BA-BATIMENT AGRICOLE-FRANCE_BOIS_ARDOISE', 'P1-BA-BATIMENT AGRICOLE-FRANCE_BOIS_TUILE']
danube_archs len of 240
danube_archs set len of 240


In [20]:
# charge archetypes from dept archs outputs
def depts_archetypes_names():
    arch_per_dept_paths = GPD_SORTING_OUTPUT_PATH / "new_territory_archs_per_dept"
    files_archs_depts = os.listdir(arch_per_dept_paths)
    print(files_archs_depts[:2])
    archs_depts = [
        "-".join(file_name.split("-")[:4]) for file_name in files_archs_depts
    ]
    print(f"archs_depts len of {len(archs_depts)}")
    print(archs_depts[:2])
    archs_depts_set = set(archs_depts)
    print(f"archs_depts_set len of {len(archs_depts_set)}\n")

    return archs_depts_set, archs_depts, files_archs_depts

archs_depts_set, archs_depts, files_archs_depts = depts_archetypes_names()

['P1-BA-BATIMENT D ENSEIGNEMENT-FRANCE_BRIQUE_TUILE-dep31-len1.csv', 'P1-BA-BATIMENT D ENSEIGNEMENT-FRANCE_BRIQUE_TUILE-dep59-len1.csv']
archs_depts len of 6973
['P1-BA-BATIMENT D ENSEIGNEMENT-FRANCE_BRIQUE_TUILE', 'P1-BA-BATIMENT D ENSEIGNEMENT-FRANCE_BRIQUE_TUILE']
archs_depts_set len of 515



In [21]:
# check territories compatibility
print("*" * 50)
territ_danu_p1 = catalogue[catalogue["NUMERO_PERIODE"] == "P1"]["TERRITOIRE"].unique()
print("territ_danu_p1 \n", len(territ_danu_p1))
for e in territ_danu_p1:
    print(e)
territ_depts_p1 = {arch.split("-")[-1] for arch in archs_depts if "P1" in arch}
print("\nterrit_depts_p1 \n", len(territ_depts_p1))
for e in territ_depts_p1:
    print(e)
print(
    "territ_depts_p1 = territ_danu_p1 ",
    sorted(territ_depts_p1) == sorted(territ_danu_p1),
)

**************************************************
territ_danu_p1 
 20
FRANCE_BOIS_ARDOISE
FRANCE_BOIS_TUILE
FRANCE_BRIQUE_ARDOISE
FRANCE
FRANCE_BRIQUE_TUILE
FRANCE_PIERRE_CALCAIRE_ARDOISE
FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_ZINC
FRANCE_TERRE_TUILE
FRANCE_PIERRE_GALET_TUILE
FRANCE_PIERRE_GNEISS_TUILE
FRANCE_PIERRE_GRANITE_ARDOISE
FRANCE_PIERRE_GRANITE_TUILE
FRANCE_PIERRE_GRES_ARDOISE
FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_MEULIERE_TUILE
FRANCE_PIERRE_SCHISTE_TUILE
FRANCE_PIERRE_VOLCANIQUE_ARDOISE
FRANCE_PIERRE_ARDOISE
FRANCE_PIERRE_VOLCANIQUE_TUILE

territ_depts_p1 
 17
FRANCE_PIERRE_SCHISTE_TUILE
FRANCE_PIERRE_GRANITE_TUILE
FRANCE_BOIS_ARDOISE
FRANCE_PIERRE_SCHISTE_ARDOISE
FRANCE_BOIS_TUILE
FRANCE_PIERRE_GRES_ARDOISE
FRANCE_PIERRE_VOLCANIQUE_TUILE
FRANCE_BRIQUE_ARDOISE
FRANCE_PIERRE_GRANITE_ARDOISE
FRANCE_PIERRE_CALCAIRE_ARDOISE
FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GNEISS_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_TERRE_TUILE
FRANCE_PIERRE_MEULIERE_TUILE
FRANCE_PIERRE

In [22]:
ter_dan_not_dept = sorted(list(set(territ_danu_p1) - set(territ_depts_p1)))
ter_dept_not_dan = sorted(list(set(territ_depts_p1) - set(territ_danu_p1)))

print("\nterrit_danu_p1 - territ_depts_p1 , len", len(ter_dan_not_dept))
for e in ter_dan_not_dept:
    print(e)
print("\nterrit_depts_p1 - territ_danu_p1 , length", len(ter_dept_not_dan))
for e in ter_dept_not_dan:
    print(e)

print(
    "\nall different territories P1 has length ",
    len(list(set(territ_danu_p1).symmetric_difference(set(territ_depts_p1)))),
    "\n ",
)


territ_danu_p1 - territ_depts_p1 , len 4
FRANCE
FRANCE_PIERRE_ARDOISE
FRANCE_PIERRE_GALET_TUILE
FRANCE_PIERRE_VOLCANIQUE_ARDOISE

territ_depts_p1 - territ_danu_p1 , length 1
FRANCE_PIERRE_SCHISTE_ARDOISE

all different territories P1 has length  5 
 


In [11]:
[arc for arc in archs_depts if "FRANCE_PIERRE_MEULIERE_ARDOISE" in arc]

[]

In [12]:
for ter in ter_dan_not_dept:
    print(ter)
    display(catalogue[catalogue["TERRITOIRE"] == ter])

FRANCE


,NUMERO_TYPOLOGIE,NUMERO_PERIODE,DISPOSITIF_MUR_OPTION1,DISPOSITIF_MUR_OPTION2,DISPOSITIF_MUR_OPTION3,DISPOSTIIF_TOIT_ OPTION1,DISPOSITIF_TOIT_OPTION2,TERRITOIRE,COUVERTURE_ROUTE,VENTILATION,...,POURCENTAGE_VITRAGE,PROTECTIONS_SOLAIRES,TYPE_VITRAGE,USAGE,NOM_TYPOLOGIE,PHOTO,source photo,PERMEABILITE,PLANCHER,archetype
3,AGR_BA_P1_TBR_TF,P1,PAV_M_P1_TBr_br,ND,ND,HAB_T_P1_TBR_TU,ND,FRANCE,ASPHALT,NON,...,0,NON,NON,BATIMENT AGRICOLE,BA,NaN,NaN,9,TERRE BATTUE1NON0,P1-BA-BATIMENT AGRICOLE-FRANCE
18,AGR_BA_P2_TF,P2,BA_M_P2_TN_IND_TOL,ND,ND,TER_T_P2_TN_BAC,HAB_T_P2_TBR_TC,FRANCE,ASPHALT,NON,...,10,NON,SIMPLE VITRAGE,BATIMENT AGRICOLE,BA,NaN,NaN,8,DALLE BETON SUR TERRE PLEIN8NON0,P2-BA-BATIMENT AGRICOLE-FRANCE
19,AGR_BA_P3_TF,P3,BA_M_P3_TN_IND_TOL,ND,ND,TER_T_P3_TN_BAC,ND,FRANCE,ASPHALT,NON,...,10,NON,SIMPLE VITRAGE,BATIMENT AGRICOLE,BA,NaN,NaN,8,DALLE BETON SUR TERRE PLEIN8NON0,P3-BA-BATIMENT AGRICOLE-FRANCE
20,AGR_BA_P4_TF,P4,BA_M_P4_TN_PS,ND,ND,TER_T_P4_TN_BAC,ND,FRANCE,ASPHALT,NON,...,10,NON,SIMPLE VITRAGE,BATIMENT AGRICOLE,BA,NaN,NaN,6,DALLE BETON SUR TERRE PLEIN8NON0,P4-BA-BATIMENT AGRICOLE-FRANCE
21,AGR_BA_P5_TF,P5,BA_M_P5_TN_PS,ND,ND,TER_T_P5_TN_BAC,ND,FRANCE,ASPHALT,NON,...,10,NON,SIMPLE VITRAGE,BATIMENT AGRICOLE,BA,NaN,NaN,5,DALLE BETON SUR TERRE PLEIN8NON0,P5-BA-BATIMENT AGRICOLE-FRANCE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,TER_I_P7_TF,P7,IMM_M_P7_TN_TER_MR,IMM_M_P7_TN_TER_PAR,IMM_M_P7_TN_TER_BE,HAB_T_P7_TN,ND,FRANCE,ASPHALT,OUI,...,40,CASQUETTE,TRIPLE VITRAGE A ISOLATION RENFORCEE,TERTIAIRE,I,NaN,NaN,2,DALLE BETON SOUS SOL20DALLE BETON15,P7-I-TERTIAIRE-FRANCE
231,TER_IGH_P4_TF,P4,BGH_M_P4_TN_TER_MR,ND,ND,HAB_T_P4_TN_GR,ND,FRANCE,ASPHALT,NON,...,50,STORE INTERIEUR,DOUBLE VITRAGE,TERTIAIRE,IGH,NaN,NaN,5,DALLE BETON SOUS SOL20DALLE BETON15,P4-IGH-TERTIAIRE-FRANCE
232,TER_IGH_P5_TF,P5,BGH_M_P5_TN_TER_MR,ND,ND,HAB_T_P5_TN_GR,ND,FRANCE,ASPHALT,OUI,...,60,STORE INTERIEUR,DOUBLE VITRAGE A ISOLATION RENFORCEE,TERTIAIRE,IGH,NaN,NaN,4,DALLE BETON SOUS SOL20DALLE BETON15,P5-IGH-TERTIAIRE-FRANCE
233,TER_IGH_P6_TF,P6,BGH_M_P6_TN_TER_MR,ND,ND,HAB_T_P6_TN_GR,ND,FRANCE,ASPHALT,OUI,...,70,STORE INTERIEUR,DOUBLE VITRAGE A ISOLATION RENFORCEE,TERTIAIRE,IGH,NaN,NaN,3,DALLE BETON SOUS SOL20DALLE BETON15,P6-IGH-TERTIAIRE-FRANCE


FRANCE_PIERRE_ARDOISE


,NUMERO_TYPOLOGIE,NUMERO_PERIODE,DISPOSITIF_MUR_OPTION1,DISPOSITIF_MUR_OPTION2,DISPOSITIF_MUR_OPTION3,DISPOSTIIF_TOIT_ OPTION1,DISPOSITIF_TOIT_OPTION2,TERRITOIRE,COUVERTURE_ROUTE,VENTILATION,...,POURCENTAGE_VITRAGE,PROTECTIONS_SOLAIRES,TYPE_VITRAGE,USAGE,NOM_TYPOLOGIE,PHOTO,source photo,PERMEABILITE,PLANCHER,archetype
146,REL_I_P1_TF,P1,IMM_M_P1_TPI,ND,ND,HAB_T_P1_TPI_AR,ND,FRANCE_PIERRE_ARDOISE,ASPHALT,NON,...,5,EPAISSEUR MUR,SIMPLE VITRAGE,BATIMENT RELIGIEUX,I,NaN,NaN,9,REVETEMENT SUR TERRE PLEIN4NON0,P1-I-BATIMENT RELIGIEUX-FRANCE_PIERRE_ARDOISE
155,SAN_I_P1_TAR,P1,IMM_M_P1_TPI,ND,ND,HAB_T_P1_TPI_AR,ND,FRANCE_PIERRE_ARDOISE,ASPHALT,NON,...,25,EPAISSEUR MUR,SIMPLE VITRAGE,BATIMENT DE SANTE,I,NaN,NaN,9,DALLE BETON SOUS SOL20PLANCHER BOIS3,P1-I-BATIMENT DE SANTE-FRANCE_PIERRE_ARDOISE


FRANCE_PIERRE_GALET_TUILE


,NUMERO_TYPOLOGIE,NUMERO_PERIODE,DISPOSITIF_MUR_OPTION1,DISPOSITIF_MUR_OPTION2,DISPOSITIF_MUR_OPTION3,DISPOSTIIF_TOIT_ OPTION1,DISPOSITIF_TOIT_OPTION2,TERRITOIRE,COUVERTURE_ROUTE,VENTILATION,...,POURCENTAGE_VITRAGE,PROTECTIONS_SOLAIRES,TYPE_VITRAGE,USAGE,NOM_TYPOLOGIE,PHOTO,source photo,PERMEABILITE,PLANCHER,archetype
9,AGR_BA_P1_TGAL_TUI,P1,PAV_M_P1_TPi_Gal,ND,ND,HAB_T_P1_TBR_TC,ND,FRANCE_PIERRE_GALET_TUILE,ASPHALT,NON,...,0,NON,NON,BATIMENT AGRICOLE,BA,NaN,NaN,9,TERRE BATTUE1NON0,P1-BA-BATIMENT AGRICOLE-FRANCE_PIERRE_GALET_TUILE
32,CHA_I_P1_TGAL_TUI,P1,IMM_M_P1_TPi_Gra,ND,ND,HAB_T_P1_TBR_TP,ND,FRANCE_PIERRE_GALET_TUILE,ASPHALT,NON,...,20,EPAISSEUR MUR,SIMPLE VITRAGE,CHATEAU,I,NaN,NaN,8,VOUTE PIERRE SOUS SOL40BOIS3,P1-I-CHATEAU-FRANCE_PIERRE_GALET_TUILE
59,HAB_I_P1_TGAL_TUI,P1,PAV_M_P1_TPi_Gal,ND,ND,HAB_T_P1_TBR_TC,ND,FRANCE_PIERRE_GALET_TUILE,ASPHALT,NON,...,17,EPAISSEUR MUR_VOLET,SIMPLE VITRAGE,HABITAT,I,NaN,NaN,8,REVETEMENT SUR TERRE PLEIN4PLANCHER BOIS3,P1-I-HABITAT-FRANCE_PIERRE_GALET_TUILE
101,HAB_P_P1_TGAL_TUI,P1,PAV_M_P1_TPi_Gal,ND,ND,HAB_T_P1_TBR_TC,ND,FRANCE_PIERRE_GALET_TUILE,ASPHALT,NON,...,17,EPAISSEUR MUR_VOLET,SIMPLE VITRAGE,HABITAT,P,NaN,NaN,9,REVETEMENT SUR TERRE PLEIN4NON0,P1-P-HABITAT-FRANCE_PIERRE_GALET_TUILE
216,TER_I_P1_TGAL_TUI,P1,PAV_M_P1_TPi_Gal,ND,ND,HAB_T_P1_TBR_TC,ND,FRANCE_PIERRE_GALET_TUILE,ASPHALT,NON,...,17,EPAISSEUR MUR_VOLET,SIMPLE VITRAGE,TERTIAIRE,I,NaN,NaN,8,REVETEMENT SUR TERRE PLEIN4PLANCHER BOIS3,P1-I-TERTIAIRE-FRANCE_PIERRE_GALET_TUILE


FRANCE_PIERRE_VOLCANIQUE_ARDOISE


,NUMERO_TYPOLOGIE,NUMERO_PERIODE,DISPOSITIF_MUR_OPTION1,DISPOSITIF_MUR_OPTION2,DISPOSITIF_MUR_OPTION3,DISPOSTIIF_TOIT_ OPTION1,DISPOSITIF_TOIT_OPTION2,TERRITOIRE,COUVERTURE_ROUTE,VENTILATION,...,POURCENTAGE_VITRAGE,PROTECTIONS_SOLAIRES,TYPE_VITRAGE,USAGE,NOM_TYPOLOGIE,PHOTO,source photo,PERMEABILITE,PLANCHER,archetype
17,AGR_BA_P1_TVOL_ARD,P1,PAV_M_P1_TPi_Vol,ND,ND,HAB_T_P1_TPI_AR,ND,FRANCE_PIERRE_VOLCANIQUE_ARDOISE,ASPHALT,NON,...,0,NON,NON,BATIMENT AGRICOLE,BA,NaN,NaN,9,TERRE BATTUE1NON0,P1-BA-BATIMENT AGRICOLE-FRANCE_PIERRE_VOLCANIQ...
38,CHA_I_P1_TVOL_ARD,P1,IMM_M_P1_TPi_Vol,ND,ND,HAB_T_P1_TPI_AR,ND,FRANCE_PIERRE_VOLCANIQUE_ARDOISE,ASPHALT,NON,...,20,EPAISSEUR MUR,SIMPLE VITRAGE,CHATEAU,I,NaN,NaN,8,VOUTE PIERRE SOUS SOL40BOIS3,P1-I-CHATEAU-FRANCE_PIERRE_VOLCANIQUE_ARDOISE
67,HAB_I_P1_TVOL_ARD,P1,IMM_M_P1_TPi_Vol,ND,ND,HAB_T_P1_TPI_AR,ND,FRANCE_PIERRE_VOLCANIQUE_ARDOISE,ASPHALT,NON,...,17,EPAISSEUR MUR_VOLET,SIMPLE VITRAGE,HABITAT,I,NaN,NaN,8,REVETEMENT SUR TERRE PLEIN4PLANCHER BOIS3,P1-I-HABITAT-FRANCE_PIERRE_VOLCANIQUE_ARDOISE
109,HAB_P_P1_TVOL_ARD,P1,PAV_M_P1_TPi_Vol,ND,ND,HAB_T_P1_TPI_AR,ND,FRANCE_PIERRE_VOLCANIQUE_ARDOISE,ASPHALT,NON,...,17,EPAISSEUR MUR_VOLET,SIMPLE VITRAGE,HABITAT,P,NaN,NaN,9,REVETEMENT SUR TERRE PLEIN4NON0,P1-P-HABITAT-FRANCE_PIERRE_VOLCANIQUE_ARDOISE
224,TER_I_P1_TVOL_ARD,P1,IMM_M_P1_TPi_Vol,ND,ND,HAB_T_P1_TPI_AR,ND,FRANCE_PIERRE_VOLCANIQUE_ARDOISE,ASPHALT,NON,...,17,EPAISSEUR MUR_VOLET,SIMPLE VITRAGE,TERTIAIRE,I,NaN,NaN,8,REVETEMENT SUR TERRE PLEIN4PLANCHER BOIS3,P1-I-TERTIAIRE-FRANCE_PIERRE_VOLCANIQUE_ARDOISE


In [ ]:


print("#" * 50)
archs_just_danube = []
for ter in ter_dan_not_dept[1:]:
    arch_just_dan = catalogue[catalogue["TERRITOIRE"] == ter]["archetype"]
    archs_just_danube.append(arch_just_dan)
    print("\nterritory just Danube", ter, "archetypes")
    for arch in arch_just_dan:
        print(arch)
print("#" * 50)

ter_pierre_and_ardoise = [
    ter for ter in territ_danu_p1 if ("PIERRE" in ter) and ("ARDOISE" in ter)
]
print("ter_pierre_and_ardoise :", ter_pierre_and_ardoise)

files_ter_dept_not_danu = []
for ter_just_dept in ter_dept_not_dan:
    files_1dept_not_dan = [file for file in files_archs_depts if ter_just_dept in file]
    print(
        "\nter_just_dept ",
        ter_just_dept,
        " len ",
        len(files_1dept_not_dan),
        ", associated files:",
    )
    for file in files_1dept_not_dan:
        print(file)
    files_ter_dept_not_danu.append(files_1dept_not_dan)
ter_dept_not_dan_joined_archs = [
    element for sublist in files_ter_dept_not_danu for element in sublist
]
len_files_ter_dept_not_danu = [
    int(ele.split("-")[-1][3:-4]) for ele in ter_dept_not_dan_joined_archs
]
print("len_files_ter_dept_not_danu: ", sum(len_files_ter_dept_not_danu))


# there is a problem between the territories defined in the QGIS file from mapuce territories and Danube definitions.
# we need to find the correspondence between them to be able to join the archetypes in the whole French territory.
territ_danu_p2plus = catalogue[catalogue["NUMERO_PERIODE"] != "P1"]["TERRITOIRE"].unique()
print("territ_danu_p2plus \n", territ_danu_p2plus)

territ_depts_p2plus = {arch.split("-")[-1] for arch in archs_depts if "P1" not in arch}
print("territ_depts_p2plus \n", territ_depts_p2plus, "\n")

print(
    f"{(set(territ_danu_p2plus) - territ_depts_p2plus)} are not in the dept territories, but are in Danube"
)
print("*" * 50)
# check how many danube archetypes are divided into
# having territories FRANCE_BRIQUE_TUILE in Dabube is a problem,
# cause this was not defined on the initial shapefile with the territories for joining in QGIS
# how to solve this problem ?
danube_groupedby_ter_per = catalogue.groupby(["TERRITOIRE", "NUMERO_PERIODE"]).count()[
    "archetype"
]
print(danube_groupedby_ter_per)

# check the cases in Danube that are not in depts
# FRANCE_BRIQUE_TUILE
archs_danu_territ_fbt = {arch for arch in archs_danube if "FRANCE_BRIQUE_TUILE" in arch}
print(f"len archs_danu_territ_fbt is {len(archs_danu_territ_fbt)}")
for e in sorted(archs_danu_territ_fbt):
    print(e)

# general FRANCE
archs_danu_territ_all_france = {arch for arch in archs_danube if "FRANCE_" not in arch}
print(f"len archs_danu_territ_all_france is {len(archs_danu_territ_all_france)}")
archs_danu_territ_all_france_3entries = [
    arch[:-7] for arch in archs_danu_territ_all_france
]
check_unicity_all_france = []
p1_p7_1france = []
p1_multiple_france = []
p2plus_2france = []
p2plus_3france = []

i = 1
j = 1
for arch_france in sorted(archs_danu_territ_all_france):
    list_one_arch_france = [arch for arch in archs_danube if arch_france in arch]
    # print(i)
    # print(arch_france)
    # print(list_one_arch_france)
    # print("*" * 50)
    i += 1
    if len(list_one_arch_france) == 1:
        p1_p7_1france.append(list_one_arch_france)
    else:
        print(arch_france, len(list_one_arch_france))
        print(list_one_arch_france)
        print("*" * 50)

        if not arch_france.startswith("P1"):
            if len(list_one_arch_france) == 2:
                p2plus_2france.append(list_one_arch_france)
            else:
                p2plus_3france.append(list_one_arch_france)
                j += 1
        else:
            p1_multiple_france.append(list_one_arch_france)

print(j)

# if len(list_one_arch_france) > 1:
#     list_one_arch_france.remove(arch_france)
#     check_unicity_all_france += list_one_arch_france
print(
    f"check_unicity_all_france len is of {len(check_unicity_all_france)} "
    f"- all these elements are present even though there is already the territory france defined"
)

print(f"\n p1_p7_1france len {len(p1_p7_1france)}")
for e in sorted(p1_p7_1france):
    print(e)
print(f"\n p2plus_2france len {len(p2plus_2france)}")
for e in sorted(p2plus_2france):
    print(e)
print(f"\n p2plus_3france len {len(p2plus_3france)}")
for e in sorted(p2plus_3france):
    print(e)
print(f"\n p1_multiple_france len {len(p1_multiple_france)}")
for e in sorted(p1_multiple_france):
    print(len(e), e)
# in many cases there are territories in Danube that are defined as France, but there is another smaller level for it.
# this can be a problem to define the limits of the archetypes, since their territorial limit is not clearly defined.
# what exactly means territory France? In the QGIS territories there was no France brique tuile (neither France of course).
# The dificulty is in defining the territorry for those cases
# France is not always for the whole country, but the rest of the country
# if there is another archetype of the same first entries
# France brique tuile is probably in the territory of france tuile, but where exactly?

# In the following cases for the period P2+ there is 3 defined territories, but one of them is France.
# If FrAr, FrTu and FrBrTu are exhaustive categories, France should not be used in case there are 3 description.
# It is not a consistent nomenclature. France should be used if it corresponds to the whole territory
# or to the ensemble of at least 2 territories.
# Ex: 20 P2-I-HABITAT-FRANCE
# ['P2-I-HABITAT-FRANCE_ARDOISE', 'P2-I-HABITAT-FRANCE_TUILE', 'P2-I-HABITAT-FRANCE']
# Shouldn't it be FrBrTu instead of France ?
# Ex: 23 P2-P-HABITAT-FRANCE
# ['P2-P-HABITAT-FRANCE_ARDOISE', 'P2-P-HABITAT-FRANCE_BRIQUE_TUILE', 'P2-P-HABITAT-FRANCE']
# Shouldn't it be FrTu instead of France ? etc...


concat_archs_folder = GPD_SORTING_OUTPUT_PATH / "new_territory_archs_concat"
concat_archs_files = os.listdir(concat_archs_folder)
print(concat_archs_files[:2])


test = "P3-I-TERTIAIRE-FRANCE"
p1_p7_1france = [element for sublist in p1_p7_1france for element in sublist]

concat_need_1france = []

for france1 in p1_p7_1france:
    concat_need = [arch for arch in concat_archs_files if france1 in arch]
    print(f"\n All france1 - {france1} has {len(concat_need)} archetypes to concat:")
    for e in concat_need:
        print(e)
    concat_need_1france.append(concat_need)

concat_need_1france_ = [e for e in concat_need_1france if len(e) > 1]
a = 1
